The best score I got was with Polynomial features creating interactions on ALL the continuous and binary vars. This made more than 50k columns. The score on Kaggle for the elastic net regression was .12074, which is the lowers I've gotten so far. However, this drastically increased training time for almost all the models. 

I think if I had more domain knowledge, I would only create the interactions that I think would matter. But for this exercise, creating all the possible interactions seems like a better "shotgun" approach. 

# Imports

In [6]:
%load_ext autoreload
%autoreload 2
import patsy
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',100)
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_validate, GridSearchCV
import statsmodels.api as sm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from sklearn.model_selection import cross_validate
df_raw = pd.read_csv('train.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import xgboost

Notes:
- Year Sold and Built not useful by itself, need to be changed into Age at Sale
- Year Sold minus Year Remodel gives time since last remodel
- MSSubClass will need some encoding (one-hot)
- MSZoning will need encoding
- Street can be one hot encoded
- Alley will be encoded
- LotShape encoded
- LandContour encoded
- Utilities encode

## Data Transforms

In [181]:
df = df_raw.copy()

In [182]:
df['AgeHouseAtSale'] = df['YrSold'] - df['YearBuilt']

df['YearsLastRemodelAtSale']  = df['YrSold'] - df['YearRemodAdd']

df['YearsGarageBuiltAtSale'] = df['YrSold'] - df['GarageYrBlt'] 

df.drop(columns=['Id'],inplace=True)

df['MSSubClass'] = df['MSSubClass'].astype(str)

In [183]:
df['YearsLastRemodelAtSale'] = np.maximum(df['YearsLastRemodelAtSale'],0)

In [184]:
df["SqFtPerRoom"]        = df["GrLivArea"] / \
                                                        (df["TotRmsAbvGrd"] + df["FullBath"]
                                                       + df["HalfBath"] + df["KitchenAbvGr"])
df['Total_Home_Quality'] = df['OverallQual'] + df['OverallCond']
df['Total_Bathrooms']    = df['FullBath'] + 0.5 * df['HalfBath'] \
                                      + df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']
df["HighQualSF"]         = df["1stFlrSF"] + df["2ndFlrSF"]



In [185]:
df["TotalSF"] = df["1stFlrSF"] + df["2ndFlrSF"] + df["TotalBsmtSF"] 
# df["GreenArea"] = df["LotArea"] - df["GrLivArea"] - df["GarageArea"]
for var in ['1stFlrSF','2ndFlrSF','TotalBsmtSF']:
    df[var+'Pct'] = df[var]/df['TotalSF']
# for var in ['GreenArea','GrLivArea','GarageArea']:
#     df[var+'Pct'] = df[var]/df['LotArea']

In [186]:
df.describe()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,AgeHouseAtSale,YearsLastRemodelAtSale,YearsGarageBuiltAtSale,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,TotalSF,1stFlrSFPct,2ndFlrSFPct,TotalBsmtSFPct
count,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890,36.547945,22.950685,29.306019,157.307896,11.674658,2.210616,1509.619178,2567.048630,0.463508,0.128807,0.407686
std,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883,30.250152,20.639875,24.726881,28.149217,1.693527,0.785399,521.163523,821.714421,0.113344,0.154270,0.107678
min,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000,0.000000,0.000000,0.000000,83.500000,2.000000,1.000000,334.000000,334.000000,0.257544,0.000000,0.000000
25%,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000,8.000000,4.000000,6.000000,137.741071,11.000000,2.000000,1123.750000,2009.500000,0.366565,0.000000,0.333333
50%,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000,35.000000,14.000000,28.000000,154.666667,12.000000,2.000000,1458.000000,2474.000000,0.500000,0.000000,0.442645
75%,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000,54.000000,41.000000,47.000000,172.584821,13.000000,2.500000,1775.250000,3004.000000,0.500437,0.297038,0.500000
max,313.000000,215245.000000,

In [187]:
for var in df.dtypes[df.dtypes == 'int64'].index:
    if df[var].isnull().sum()>0:
        df[var+'_isnull'] = df[var].isnull().astype(int)
        df[var]=df[var].fillna(df[var].mean())

for var in df.dtypes[df.dtypes == 'float64'].index:
    if df[var].isnull().sum()>0:
        df[var+'_isnull'] = df[var].isnull().astype(int)
        df[var]=df[var].fillna(df[var].mean())

df_dummies = pd.get_dummies(df,dummy_na=True,drop_first=False)

In [188]:
df_dummies.drop(columns=df_dummies.loc[:,df_dummies.nunique() == 1].columns, inplace=True)

In [189]:
X = df_dummies.drop(columns='SalePrice')

y = np.log(df_dummies['SalePrice'])

# X['intercept'] = 1

## Creating Interaction Terms

In [15]:
polynomial_features = PolynomialFeatures(degree=2,include_bias=True)
test_df = polynomial_features.fit_transform(X)
test_df = pd.DataFrame(polynomial_features.fit_transform(X), columns = polynomial_features.get_feature_names_out(X.columns))
from sklearn.base import BaseEstimator, TransformerMixin

class PolynomialFeaturesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2, include_bias=True): # no *args or **kargs
        self.init = True
        self.degree = degree
        self.include_bias=include_bias
        self.polynomial_features = PolynomialFeatures(degree=degree,include_bias=include_bias)
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        test_df = self.polynomial_features.fit_transform(X)
        return  pd.DataFrame(self.polynomial_features.fit_transform(X), columns = self.polynomial_features.get_feature_names_out(X.columns))
polynomial_features = PolynomialFeaturesTransformer(degree=2,include_bias=True)
polynomial_features.fit(X)


PolynomialFeaturesTransformer()

### Testing Patsy

## Creating own scaler

Based on taking the log first (adding a 1 so that 0s become 1s), and then centering and dividing out the standard deviation

The log standard is incredibly promising for Elastic Net. In fact, it's almost the best performing model yet (only beaten by the stacking regression)

Bayesian Ridge also showed improvement

In [190]:
from sklearn.base import BaseEstimator, TransformerMixin

class LogStandardScaler(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        self.init = True
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        temp = np.log(X+1)
        X_scaled = (temp - temp.mean())/temp.std()
        return X_scaled.fillna(0)


In [191]:
test_scaler = LogStandardScaler()

# Training

## SKLEARN

### Lasso

In [26]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

In [27]:
lasso_cv = LassoCV(cv=5, random_state=0,max_iter=20000)

In [29]:
lasso_pipeline = make_pipeline(PolynomialFeaturesTransformer()
                                ,LogStandardScaler()
                               ,lasso_cv)
lasso_pipeline.fit(X=X,y=y)
lasso_cv.alpha_
# test = pd.DataFrame(lasso_pipeline.coef_,index=X.columns)
# test.loc[test[0]>0]

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


KeyboardInterrupt: 

### Elastic Net
With all of these interaction variables, fitting the CV for Elastic Net takes much longer (>6 minutes on my comp)
I think this has to do with having too many variables compared to observations, creating longer convergence times

In [24]:
from sklearn.linear_model import ElasticNetCV

In [87]:
# elastic_net_cv = ElasticNetCV(l1_ratio=[0,.5,1]
#                              ,alphas=[0.1,1,10,100]
#                              )
elastic_net_cv = ElasticNetCV(cv=5, random_state=0, l1_ratio=[0.5,0.7,0.8,0.9,1],max_iter=20000,n_jobs=-1)

In [88]:
elastic_net_pipeline = make_pipeline(
    # PolynomialFeaturesTransformer(),
LogStandardScaler(),elastic_net_cv)

In [89]:
elastic_net_pipeline.fit(X=X,y=y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('elasticnetcv',
                 ElasticNetCV(cv=5, l1_ratio=[0.5, 0.7, 0.8, 0.9, 1],
                              max_iter=20000, n_jobs=-1, random_state=0))])

In [90]:
elastic_net_cv.l1_ratio_

0.5

In [91]:
elastic_net_cv.alpha_

0.00794428509169608

In [96]:
pd.DataFrame(elastic_net_cv.coef_,index=X.columns).sort_values(by=0,ascending=False).head(20)

,0
GrLivArea,0.078381
TotalSF,0.077419
OverallQual,0.056143
LotArea,0.037983
OverallCond,0.036791
GarageCars,0.035062
YearBuilt,0.031387
BsmtFinSF1,0.019519
KitchenQual_Ex,0.019459
Neighborhood_Crawfor,0.018638


In [192]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(l1_ratio=0.5,alpha=0.00794428509169608,max_iter=20000)
elastic_net_pipeline = make_pipeline(
    PolynomialFeaturesTransformer()
                                     ,LogStandardScaler()
                                     ,elastic_net)
elastic_net_pipeline.fit(X=X,y=y)

Pipeline(steps=[('polynomialfeaturestransformer',
                 PolynomialFeaturesTransformer()),
                ('logstandardscaler', LogStandardScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.00794428509169608, max_iter=20000))])

### Decision Tree

In [ ]:
params_dt={'splitter':['random', 'best']
        ,'min_samples_split':[2, 3, 4, 5, 6, 8, 10]
        ,'min_samples_leaf':[0.01, 0.02, 0.03, 0.04]
        ,'min_impurity_decrease':[0.0, 0.0005, 0.005, 0.05, 0.10, 0.15, 0.2]
        ,'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None]
        ,'max_features':[ 0.95, 0.90, 0.85, 0.80, 0.75, 0.70]
        ,'max_depth':[None, 2,4,6,8]
        ,'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01, 0.05]}
        
decision_tree = DecisionTreeRegressor()
bayes_search_dt = BayesSearchCV(decision_tree,params_dt)

In [47]:
decision_tree = DecisionTreeRegressor(max_depth=6, max_features=0.85, max_leaf_nodes=40,
                      min_samples_leaf=0.02, min_samples_split=6,
                      min_weight_fraction_leaf=0.0075)

In [48]:
decision_tree_pipeline = make_pipeline(LogStandardScaler(),decision_tree)

In [49]:
decision_tree_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=6, max_features=0.85,
                                       max_leaf_nodes=40, min_samples_leaf=0.02,
                                       min_samples_split=6,
                                       min_weight_fraction_leaf=0.0075))])

## XGBoost

In [156]:
xg_boost = xgboost.XGBRegressor(n_estimators=800,learning_rate=0.05)
xg_boost_pipeline = make_pipeline(
    # LogStandardScaler(),

xg_boost)

In [157]:
params_xg = {
    # 'xgbregressor__n_estimators':[400,600,800],
# 'xgbregressor__learning_rate':[0.01,0.05,0.1]
    'xgbregressor__max_depth':[2, 3, 5],
    # 'xgbregressor__gamma':[0],
    'xgbregressor__min_child_weight':[1, 2, 3, 4],
    # 'xgbregressor__subsample':[0.5, 0.7, 0.8, 0.9, 1],
    'xgbregressor__colsample_bytree':[0.3 ,0.5, 0.7],
}

In [174]:
grid_search_xg = GridSearchCV(xg_boost_pipeline,params_xg)

In [175]:
grid_search_xg.fit(X,y)

GridSearchCV(estimator=Pipeline(steps=[('logstandardscaler',
                                        LogStandardScaler()),
                                       ('xgbregressor',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     early_stopping_rounds=None,
                                                     enable_categorical=False,
                                                     eval_metric=None,
                                                     gamma=None, gpu_id=None,
                                                     grow_policy=None,
      

In [176]:
grid_search_xg.best_params_

{'xgbregressor__colsample_bytree': 0.7,
 'xgbregressor__max_depth': 3,
 'xgbregressor__min_child_weight': 4}

In [194]:
xg_boost = xgboost.XGBRegressor(n_estimators=800,learning_rate=0.05, colsample_bytree = 0.7, max_depth = 3, min_child_weight=4)
xg_boost_pipeline = make_pipeline(
    # LogStandardScaler(),
xg_boost)
xg_boost_pipeline.fit(X,y)

Pipeline(steps=[('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.7, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.05, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=3, max_leaves=0, min_child_weight=4,
                              missing=nan, monotone_constraints='()',
                              n_estimators=800, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=1, ...))])

## Random Forest
Fit time for Random Forests also increased drastically

In [49]:
from sklearn.ensemble import RandomForestRegressor

In [202]:
random_forest_initial = RandomForestRegressor(n_estimators=600)

In [194]:
random_forest_pipeline = make_pipeline(LogStandardScaler(),random_forest_initial)

In [195]:
random_forest_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(n_estimators=600, oob_score=True))])

In [243]:
params_rf={
        ## 'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01, 0.05],
        ##   'min_samples_split': [0.01, 0.02, 0.03, 0.04, 0.06, 0.08, 0.1,2],
        ##    'min_samples_leaf':[1,2,4,6,8,10,20,30],
        ##   'min_impurity_decrease':[0.0, 0.01, 0.05, 0.10, 0.15, 0.2],
        ##   'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
         #   'max_features':[0.8, 0.7, 0.6, 0.5, 0.4, 'auto'],
           'max_depth':[None,2,4,6,8,10,20],
           }
        

In [244]:
bayes_search_rf = GridSearchCV(random_forest_initial,params_rf,n_jobs=-1)

In [245]:
bayes_search_rf.fit(X,y)

GridSearchCV(estimator=RandomForestRegressor(n_estimators=600), n_jobs=-1,
             param_grid={'max_depth': [None, 2, 4, 6, 8, 10, 20]})

In [246]:
## Doesn't seem like I can really get better than just the default parameters
bayes_search_rf.best_params_

{'max_depth': None}

In [50]:
random_forest = RandomForestRegressor(max_depth=None, max_features=0.4, 
                      n_estimators=600, oob_score=True)

In [51]:
random_forest.fit(X,y)

RandomForestRegressor(max_features=0.4, n_estimators=600, oob_score=True)

## Adaboost

In [96]:
from sklearn.ensemble import AdaBoostRegressor

In [62]:
adaboost = AdaBoostRegressor()

In [63]:
adaboost_pipeline = make_pipeline(LogStandardScaler(),adaboost)

In [64]:
adaboost_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('adaboostregressor', AdaBoostRegressor())])

## Light GBM
Light GBM breaks because of the Patsy naming convention (creating interactions with : in the middle)

In [97]:
from lightgbm import LGBMRegressor

In [160]:
light_gbm = LGBMRegressor(n_estimators=800,learning_rate=0.05 ,max_depth=3, num_leaves=5, min_child_weight=0.00001, subsample=0.3, colsample_bytree = 0.3, lambda_l1 = 0.1)

light_gbm_pipeline = make_pipeline(
    # LogStandardScaler(),

light_gbm)

light_gbm_pipeline.fit(X,y)

[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1


Pipeline(steps=[('lgbmregressor',
                 LGBMRegressor(colsample_bytree=0.3, lambda_l1=0.1,
                               learning_rate=0.05, max_depth=3,
                               min_child_weight=1e-05, n_estimators=800,
                               num_leaves=5, subsample=0.3))])

In [99]:
light_gbm_pipeline.steps

[('logstandardscaler', LogStandardScaler()),
 ('lgbmregressor', LGBMRegressor())]

In [137]:
params_light = {
    # 'lgbmregressor__n_estimators' : [100,200,400,800]
                # ,'lgbmregressor__learning_rate' : [0.01,0.05,0.1,0.2]
            #  'lgbmregressor__max_depth':[2, 3, 5, -1],
            # 'lgbmregressor__num_leaves': [5,11, 21, 31,],
            # 'lgbmregressor__min_child_weight':[0.00001, 0.0001,0.001, 0.01, 0.1, 1],
            # 'lgbmregressor__subsample':[0.3, 0.4, 0.5, 0.7, 1],
            # 'lgbmregressor__colsample_bytree':[0.3 ,0.5, 0.7],
            # 'lgbmregressor__lambda_l1': [0,0.1,1,10,100]
            # 'lgbmregressor__lambda_l2': [0,0.1,1,10,100]
                }
light_grid = GridSearchCV(light_gbm_pipeline,params_light)
light_grid.fit(X,y)

[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current

GridSearchCV(estimator=Pipeline(steps=[('logstandardscaler',
                                        LogStandardScaler()),
                                       ('lgbmregressor',
                                        LGBMRegressor(colsample_bytree=0.3,
                                                      learning_rate=0.05,
                                                      max_depth=3,
                                                      min_child_weight=1e-05,
                                                      n_estimators=800,
                                                      num_leaves=5,
                                                      subsample=0.3))]),
             param_grid={'lgbmregressor__lambda_l2': [0, 0.1, 1, 10, 100]})

In [138]:
light_grid.best_params_

{'lgbmregressor__lambda_l2': 0.1}

## GradientBoostingRegressor

In [54]:
from sklearn.ensemble import GradientBoostingRegressor

In [162]:
gbr = GradientBoostingRegressor(n_estimators=800, learning_rate=0.05, max_leaf_nodes=30, max_features=0.4, max_depth=3, subsample=0.8)

gbr_pipeline = make_pipeline(
    # LogStandardScaler(),
    gbr)

gbr.fit(X,y)

GradientBoostingRegressor(learning_rate=0.05, max_features=0.4,
                          max_leaf_nodes=30, n_estimators=800, subsample=0.8)

In [87]:
params_gbr = {
    # 'gradientboostingregressor__n_estimators' : [100,200,400,800]
                # ,'gradientboostingregressor__learning_rate' : [0.01,0.05,0.1,0.2]
                ##'gradientboostingregressor__min_samples_split':[2,10,20,50]
                ## 'gradientboostingregressor__min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01, 0.05],
        ##    'gradientboostingregressor__min_samples_leaf':[1,2,4,6,8,10,20,30],
        ##   'gradientboostingregressor__min_impurity_decrease':[0.0, 0.01, 0.05, 0.10, 0.15, 0.2],
        #    'gradientboostingregressor__max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
        #    'gradientboostingregressor__max_features':[0.8, 0.7, 0.6, 0.5, 0.4, 1],
            #  'gradientboostingregressor__max_depth':[2, 3, 5],
            # 'gradientboostingregressor__subsample':[0.5, 0.7, 0.8, 0.9, 1],
                }
gbr_grid = GridSearchCV(gbr_pipeline,params_gbr)
gbr_grid.fit(X,y)


GridSearchCV(estimator=Pipeline(steps=[('logstandardscaler',
                                        LogStandardScaler()),
                                       ('gradientboostingregressor',
                                        GradientBoostingRegressor(learning_rate=0.05,
                                                                  max_features=0.4,
                                                                  max_leaf_nodes=30,
                                                                  n_estimators=800))]),
             param_grid={'gradientboostingregressor__subsample': [0.5, 0.7, 0.8,
                                                                  0.9, 1]})

In [88]:
gbr_grid.best_params_

{'gradientboostingregressor__subsample': 0.8}

## Stochastic Gradient Descent Regression

In [69]:
from sklearn.linear_model import SGDRegressor

In [70]:
SGDR = SGDRegressor()

SGDR_pipeline = make_pipeline(LogStandardScaler(),SGDR)

SGDR_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('sgdregressor', SGDRegressor())])

## from sklearn.svm import SVR
Support Vector Machine

In [71]:
from sklearn.svm import SVR

In [72]:
svr = SVR()

svr_pipeline = make_pipeline(LogStandardScaler(),svr)

svr_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()), ('svr', SVR())])

## from sklearn.linear_model import BayesianRidge

In [164]:
from sklearn.linear_model import BayesianRidge

In [171]:
bayesianr = BayesianRidge()

bayesianr_pipeline = make_pipeline(
    # PolynomialFeaturesTransformer(),
    LogStandardScaler(),
    bayesianr)

bayesianr_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('bayesianridge', BayesianRidge())])

In [177]:
bayesianr_pipeline['bayesianridge'].

SyntaxError: invalid syntax (17819437.py, line 1)

In [178]:
bayesianr.alpha_

96.69951275089076

In [179]:
bayesianr.coef_

array([ 7.45914156e-03,  3.77056559e-02,  4.06458071e-02,  3.61110875e-02,
        2.30057834e-02,  1.86658576e-02,  5.49125432e-03,  1.94884647e-02,
       -1.21363738e-03, -1.00124104e-02,  1.19653052e-02,  3.90666688e-02,
        3.06096554e-03, -2.23287602e-04,  4.87727344e-02,  1.29637398e-02,
        1.37571865e-03,  1.58681078e-02,  1.35063409e-02, -2.33553989e-03,
       -9.45272336e-03,  1.12833791e-02,  1.35021158e-02,  1.36015689e-03,
        2.76778728e-02,  1.29687363e-02,  9.11980620e-03,  4.19442535e-03,
        2.90634776e-03,  2.16097195e-03,  1.11441898e-02,  5.14792286e-03,
       -3.84483158e-03, -7.49958532e-04, -3.40519743e-03, -9.63667953e-03,
        2.02211566e-03, -3.31888884e-03,  5.01327087e-02, -1.46343493e-02,
        9.39183552e-03, -5.84109579e-04,  4.44644181e-03, -1.54130682e-03,
        1.50018742e-03,  1.50018742e-03,  1.16363757e-04, -7.33423031e-03,
        3.21245639e-03,  7.20281577e-07,  5.75158288e-03, -6.15518283e-03,
       -9.10986724e-04, -

## from sklearn.kernel_ridge import KernelRidge

In [75]:
from sklearn.kernel_ridge import KernelRidge

In [76]:
kernelr = KernelRidge()

kernelr_pipeline = make_pipeline(LogStandardScaler(),kernelr)

kernelr_pipeline.fit(X,y)

Pipeline(steps=[('logstandardscaler', LogStandardScaler()),
                ('kernelridge', KernelRidge())])

## Catboost

In [196]:
from catboost import CatBoostRegressor

In [225]:
catboost_reg = CatBoostRegressor(n_estimators=400,learning_rate=0.1,logging_level='Silent', subsample= 0.7)

In [224]:
params_cat = {
    # 'n_estimators' : [100,200,400,800]
                # ,'learning_rate' : [0.01,0.05,0.1,0.2]
            ##  'max_depth':[2, 3, 6, 10],
            ## 'num_leaves': [21, 31, 41],
            # 'lgbmregressor__min_child_weight':[0.00001, 0.0001,0.001, 0.01, 0.1, 1],
            # 'subsample':[0.3, 0.4, 0.5, 0.7, 1],
            # 'colsample_bytree':[0.3 ,0.5, 0.7],
            # 'lgbmregressor__lambda_l1': [0,0.1,1,10,100]
            # 'lgbmregressor__lambda_l2': [0,0.1,1,10,100]
}
grid_cat = GridSearchCV(catboost_reg,params_cat)
grid_cat.fit(X,y)

ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Yichen\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Yichen\AppData\Roaming\Python\Python39\site-packages\catboost\core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "C:\Users\Yichen\AppData\Roaming\Python\Python39\site-packages\catboost\core.py", line 2339, in _fit
    train_params = self._prepare_train_params(
  File "C:\Users\Yichen\AppData\Roaming\Python\Python39\site-packa

In [222]:
grid_cat.best_params_

{'subsample': 0.7}

## Ensemble

In [33]:
from sklearn.ensemble import VotingRegressor, StackingRegressor

In [143]:
voting_reg = VotingRegressor([('elastic_net',elastic_net_pipeline)
                            #   ,('rf',random_forest_pipeline)
                              ,('xg_boost',xg_boost_pipeline)
                             # ,('adaboost',adaboost_pipeline)
                                  ,('lightgbm',light_gbm_pipeline)
                                  ,('gbr',gbr_pipeline)
                                #   ,('bayesianRidge',bayesianr_pipeline)
                                  # ,('kernelRidge',kernelr)
                             ])

In [144]:
voting_reg.fit(X,y)

[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1


VotingRegressor(estimators=[('elastic_net',
                             Pipeline(steps=[('polynomialfeaturestransformer',
                                              PolynomialFeaturesTransformer()),
                                             ('logstandardscaler',
                                              LogStandardScaler()),
                                             ('elasticnet',
                                              ElasticNet(alpha=0.00794428509169608,
                                                         max_iter=20000))])),
                            ('xg_boost',
                             Pipeline(steps=[('logstandardscaler',
                                              LogStandardScaler()),
                                             ('xgbregressor',
                                              XGBRegressor(base_score=0....
                                              LGBMRegressor(colsample_bytree=0.3,
                                                            lambda_l1=0.1,
                                                            learning_rate=0.05,
                                                            max_depth=3,
                                                            min_child_weight=1e-05,
                                                            n_estimators=800,
                                                            num_leaves=5,
                                                            subsample=0.3))])),
                            ('gbr',
                             Pipeline(steps=[('logstandardscaler',
                                              LogStandardScaler()),
                                             ('gradientboostingregressor',
                                              GradientBoostingRegressor(learning_rate=0.05,
                                                                        max_features=0.4,
                                                                        max_leaf_nodes=30,
                                                                        n_estimators=800,
                                                                        subsample=0.8))]))])

In [227]:
stacking_reg = StackingRegressor([('elastic_net',elastic_net_pipeline)
                              # ,('rf',random_forest_pipeline)
                              ,('xg_boost',xg_boost_pipeline)
                             # ,('adaboost',adaboost_pipeline)
                                  ,('lightgbm',light_gbm_pipeline)
                                  ,('gbr',gbr_pipeline)
                              #     ,('bayesianRidge',bayesianr_pipeline)
                                  # ,('kernelRidge',kernelr)
                                  ,('catboost',catboost_reg)
                             ]
                            #  , final_estimator=ElasticNet(alpha=0.1)
                             ,cv= 3
                             ,final_estimator=catboost_reg
                             , passthrough=True)

In [228]:
stacking_reg.fit(X,y)

[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1


StackingRegressor(cv=3,
                  estimators=[('elastic_net',
                               Pipeline(steps=[('polynomialfeaturestransformer',
                                                PolynomialFeaturesTransformer()),
                                               ('logstandardscaler',
                                                LogStandardScaler()),
                                               ('elasticnet',
                                                ElasticNet(alpha=0.00794428509169608,
                                                           max_iter=20000))])),
                              ('xg_boost',
                               Pipeline(steps=[('xgbregressor',
                                                XGBRegressor(base_score=0.5,
                                                             booster='gbtree',
                                                             callbacks=None,
                                                             c...
                                                              subsample=0.3))])),
                              ('gbr',
                               Pipeline(steps=[('gradientboostingregressor',
                                                GradientBoostingRegressor(learning_rate=0.05,
                                                                          max_features=0.4,
                                                                          max_leaf_nodes=30,
                                                                          n_estimators=800,
                                                                          subsample=0.8))])),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x000001B72E347400>)],
                  final_estimator=<catboost.core.CatBoostRegressor object at 0x000001B72E347400>,
                  passthrough=True)

## Cross Val Score

In [97]:
val_results = cross_validate(lasso_pipeline,X,y, scoring=['neg_mean_squared_error','r2'], n_jobs=-1)
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.015797,0.003957,-0.020459,-0.018566,-0.015864,-0.013645,-0.01045


In [193]:
val_results = cross_validate(elastic_net_pipeline,X,y, scoring=['neg_mean_squared_error','r2'],n_jobs=-1)
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.016234,0.003941,-0.021535,-0.018135,-0.016469,-0.013681,-0.011352


In [82]:
val_results = cross_validate(decision_tree_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.037759,0.004289,-0.042593,-0.041169,-0.038267,-0.033431,-0.033332


In [195]:
val_results = cross_validate(xg_boost_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.014892,0.002631,-0.01758,-0.016339,-0.016314,-0.012788,-0.011438


In [196]:
val_results = cross_validate(random_forest_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.020707,0.00251,-0.024339,-0.022133,-0.019938,-0.018871,-0.018255


In [52]:
val_results = cross_validate(random_forest,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.018843,0.002904,-0.022583,-0.020186,-0.019576,-0.01642,-0.015448


In [245]:
val_results = cross_validate(adaboost_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.031226,0.003267,-0.034848,-0.033666,-0.030606,-0.030557,-0.026451


In [161]:
val_results = cross_validate(light_gbm_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1


,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.015409,0.002265,-0.017502,-0.017197,-0.016411,-0.012971,-0.012962


In [163]:
val_results = cross_validate(gbr_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.014691,0.003531,-0.019231,-0.017045,-0.014612,-0.011772,-0.010795


In [267]:
val_results = cross_validate(SGDR,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-1.904125e+33,1.031940e+33,-3.698560e+33,-1.823122e+33,-1.471204e+33,-1.377371e+33,-1.150367e+33


In [268]:
val_results = cross_validate(svr,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.041612,0.004728,-0.045493,-0.045119,-0.042029,-0.041672,-0.033746


In [168]:
val_results = cross_validate(bayesianr_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.020058,0.008451,-0.034016,-0.021742,-0.016922,-0.013957,-0.013655


In [271]:
val_results = cross_validate(kernelr_pipeline,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-577.065057,0.720064,-577.531635,-577.427471,-577.374909,-577.195793,-575.795477


In [226]:
val_results = cross_validate(catboost_reg,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.014059,0.002386,-0.016331,-0.0161,-0.014812,-0.011662,-0.011388


In [272]:
val_results = cross_validate(voting_reg,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.014984,0.002678,-0.018672,-0.016118,-0.015262,-0.013029,-0.011838


In [229]:
val_results = cross_validate(stacking_reg,X,y, scoring=['neg_mean_squared_error','r2'])
pd.DataFrame(val_results['test_neg_mean_squared_error']).describe().transpose()

[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored.

,count,mean,std,min,25%,50%,75%,max
0,5.0,-0.014305,0.002647,-0.017767,-0.015783,-0.014614,-0.011683,-0.01168


Scaling seems to have really benefitted the linear regression Elastic Net, but did not affect the tree based formulas much. 

Somehow kernalr is much worse now. 

# Prediction

In [39]:
df_predict_raw = pd.read_csv('test.csv')

In [40]:
df_predict = df_predict_raw.copy()

In [41]:
df_predict['AgeHouseAtSale'] = df_predict['YrSold'] - df_predict['YearBuilt']

df_predict['YearsLastRemodelAtSale']  = df_predict['YrSold'] - df_predict['YearRemodAdd']

df_predict['YearsGarageBuiltAtSale'] = df_predict['YrSold'] - df_predict['GarageYrBlt'] 

df_predict.drop(columns=['Id'],inplace=True)

df_predict['MSSubClass'] = df_predict['MSSubClass'].astype(str)

In [42]:
df_predict["TotalSF"] = df_predict["1stFlrSF"] + df_predict["2ndFlrSF"] + df_predict["TotalBsmtSF"] 
# df_predict["GreenArea"] = df_predict["LotArea"] - df_predict["GrLivArea"] - df_predict["GarageArea"]
for var in ['1stFlrSF','2ndFlrSF','TotalBsmtSF']:
    df_predict[var+'Pct'] = df_predict[var]/df_predict['TotalSF']
# for var in ['LotArea','GrLivArea','GarageArea']:
#     df_predict[var+'Pct'] = df_predict[var]/df_predict['GreenArea']

In [43]:
df_predict['YearsLastRemodelAtSale'] = np.maximum(df_predict['YearsLastRemodelAtSale'],0)
df_predict['AgeHouseAtSale'] = np.maximum(df_predict['AgeHouseAtSale'],0)
df_predict['YearsGarageBuiltAtSale'] = np.maximum(df_predict['YearsGarageBuiltAtSale'],0)

In [44]:
for var in df_predict.dtypes[df_predict.dtypes == 'int64'].index:
    if df_predict[var].isnull().sum()>0:
        df_predict[var+'_isnull'] = df_predict[var].isnull().astype(int)
        df_predict[var]=df_predict[var].fillna(df_predict[var].mean())

for var in df_predict.dtypes[df_predict.dtypes == 'float64'].index:
    if df_predict[var].isnull().sum()>0:
        df_predict[var+'_isnull'] = df_predict[var].isnull().astype(int)
        df_predict[var]=df_predict[var].fillna(df_predict[var].mean())

df_predict_dummies = pd.get_dummies(df_predict,dummy_na=True,drop_first=False)

In [45]:
_, df_predict_dummies = X.align(df_predict_dummies, axis=1, fill_value=0)

In [46]:
df_predict_dummies.drop(columns=df_predict_dummies.columns[~df_predict_dummies.columns.isin(X.columns)],inplace=True)

In [40]:
df_predict_dummies.describe()

,1stFlrSF,1stFlrSFPct,2ndFlrSF,2ndFlrSFPct,3SsnPorch,AgeHouseAtSale,Alley_Grvl,Alley_Pave,Alley_nan,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtCond_nan,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_nan,BsmtFinSF1,BsmtFinSF2,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType1_nan,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtFinType2_nan,BsmtFullBath,BsmtHalfBath,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtQual_nan,BsmtUnfSF,CentralAir_N,...,PoolArea,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_nan,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,ScreenPorch,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,TotalBsmtSFPct,TotalSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YearsGarageBuiltAtSale,YearsGarageBuiltAtSale_isnull,YearsLastRemodelAtSale,YrSold
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,...,1459.000000,1459.000000,1459.0,1459.000000,1459.000000,1459.0,1459.000000,1459.0,1459.0,1459.0,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.0,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,1156.534613,0.467215,325.967786,0.126564,1.794380,36.412611,0.047978,0.025360,0.926662,2.854010,0.825908,0.021247,0.039068,0.036326,0.077450,0.040439,0.039068,0.002056,0.887594,0.030843,0.135024,0.097327,0.085675,0.651816,0.030158,439.203704,52.619342,0.143249,0.082934,0.295408,0.054832,0.106237,0.288554,0.028787,0.022618,0.023989,0.013708,0.028101,0.034955,0.847841,0.028787,0.434454,0.065202,0.093900,0.036326,0.405072,0.434544,0.030158,554.294925,0.069225,...,1.744345,0.001371,0.0,0.000685,0.997944,0.0,0.988348,0.0,0.0,0.0,0.008225,0.002742,0.000685,0.004798,0.801234,0.007539,0.181631,0.002742,0.002056,0.061001,0.005483,0.008225,0.017820,0.825223,0.082248,0.030158,0.005483,0.002056,0.011652,0.002742,0.002056,0.080192,0.002742,0.862234,17.064428,0.004112,0.995888,6.385195,1046.117970,0.406221,2529.022634,0.998629,0.0,93.174777,1971.357779,1983.662783,30.191166,0.053461,24.108979,2007.769705
std,398.165820,0.120260,420.610226,0.153944,20.207842,30.431416,0.213793,0.157269,0.260780,0.829788,0.379318,0.144258,0.193823,0.187165,0.267396,0.197053,0.193823,0.045314,0.315973,0.172952,0.341866,0.296504,0.279980,0.476558,0.171080,455.111888,176.693301,0.350447,0.275876,0.456382,0.227730,0.308247,0.453245,0.167264,0.148734,0.153067,0.116316,0.165319,0.183730,0.359298,0.167264,0.530283,0.252295,0.291789,0.187165,0.491074,0.495867,0.171

## Predictions

### Lasso

In [124]:
df_predict_raw['SalePrice'] = np.exp(lasso_pipeline.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_lasso_p7.csv',index=False)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


### Elastic Net

In [102]:
df_predict_raw['SalePrice'] =  np.exp(elastic_net_pipeline.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_elasticnet_log_standardized_interaction.csv',index=False)

### Decision Tree

In [236]:
df_predict_raw['SalePrice'] =  np.exp(decision_tree_pipeline.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_decisiontree_logstandard.csv',index=False)

### XG Boost

In [189]:
df_predict_raw['SalePrice'] =  np.exp(xg_boost_pipeline.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_xgboost_logstandard_tuned.csv',index=False)

### Random Forest

In [53]:
df_predict_raw['SalePrice'] =  np.exp(random_forest.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_rf_tuned.csv',index=False)

In [201]:
df_predict_raw['SalePrice'] =  np.exp(random_forest_initial.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_rf_logstandard.csv',index=False)

### Light GBM

In [142]:
df_predict_raw['SalePrice'] =  np.exp(light_gbm_pipeline.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_light_gbm_tuned.csv',index=False)

### Gradient Boosting Regression

In [92]:
df_predict_raw['SalePrice'] =  np.exp(gbr.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_gbr_tuned.csv',index=False)

### Voting Regression

In [145]:
df_predict_raw['SalePrice'] =  np.exp(voting_reg.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_voting_reg_stand_tuned.csv',index=False)

### Stacking Regression

In [151]:
df_predict_raw['SalePrice'] =  np.exp(stacking_reg.predict(df_predict_dummies[X.columns]))

df_predict_raw[['Id','SalePrice']].to_csv('predictions_stacking_reg_tuned.csv',index=False)